# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from dataclasses import dataclass

from typing import List

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/usr/src/app
['/usr/src/app/event_data/2018-11-01-events.csv', '/usr/src/app/event_data/2018-11-02-events.csv', '/usr/src/app/event_data/2018-11-03-events.csv', '/usr/src/app/event_data/2018-11-04-events.csv', '/usr/src/app/event_data/2018-11-05-events.csv', '/usr/src/app/event_data/2018-11-06-events.csv', '/usr/src/app/event_data/2018-11-07-events.csv', '/usr/src/app/event_data/2018-11-08-events.csv', '/usr/src/app/event_data/2018-11-09-events.csv', '/usr/src/app/event_data/2018-11-10-events.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_data_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_data_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1595


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_data_new.csv</font>, located within the Workspace directory.  The event_data_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_data_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

# use `cassandra_db` docker container name as hostname
from cassandra.cluster import Cluster
cluster = Cluster(['cassandra_db'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute('''
    CREATE KEYSPACE IF NOT EXISTS cassandra_data_modeling
    WITH REPLICATION = {
        'class': 'SimpleStrategy',
        'replication_factor': 1
    }
''')

#### Set Keyspace

In [7]:
session.set_keyspace('cassandra_data_modeling')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




```
For the first query, there should be columns named `artist`, `song`, `length`, `sessionId`, `itemInSession`
without duplicate key consideration.

Before making `sessionId` and `itemInSession` as composite primary key, let's analyze its uniqueness.

Assumption: the composite primary key has one-to-one relation.
```

In [8]:
# column data class preparation & validate method

@dataclass
class Column:
    """
    A data class to hold column metadata
    
    Attributes
    ----------
    name: str
        column name
    is_primary_key: bool
        flag indicating that this column is a primary key
    """
    
    name: str
    is_primary_key: bool

def validate_column_uniqueness(__df: pd.core.frame.DataFrame, __columns: List[Column]) -> None:
    """
    Validate whether columns are unique according to data frame.
    If the uniqueness is not satisfied, AssertionError instance will be raised.
    
    
    Parameters
    ----------
    __df: pd.core.frame.DataFrame
        data frame containing data to be validated
    __columns: List[Column]
        selected columns from what we have in `__df` parameter
        
    Returns
    -------
    None
    """
    
    column_names = [column.name for column in __columns]
    primary_column_names = [column.name for column in __columns if column.is_primary_key]
    
    duplication_analysis_df = __df.groupby(
        column_names
    ).size(
    ).reset_index(
        name='counts'
    )[column_names].groupby(
        primary_column_names
    ).size(
    ).reset_index(
        name='counts'
    ).sort_values(
        by='counts',
        ascending=False
    )
    
    first_row = duplication_analysis_df.iloc[0]
    highest_counts = first_row['counts']
    
    assert highest_counts == 1, f'There are {highest_counts} duplications when columns = {column_names} with primary key of {primary_column_names}'

In [9]:
df = pd.read_csv('event_data_new.csv')

In [10]:
first_query_columns = [
    Column(column_name, is_primary_key)
    for (column_name, is_primary_key)
    in [
        ('sessionId'    , True),
        ('itemInSession', True),
        ('artist'       , False),
        ('song'         , False),
        ('length'       , False)
    ]
]

validate_column_uniqueness(df, first_query_columns)

In [11]:
session.execute('''
    CREATE TABLE IF NOT EXISTS musics_q1 (
        session_id      INT,
        item_in_session INT,
        artist          TEXT,
        song            TEXT,
        length          DECIMAL,
        PRIMARY KEY (session_id, item_in_session)
    )
''')

In [12]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''
            INSERT INTO musics_q1 (
                session_id,
                item_in_session,
                artist,
                song,
                length
            )
            VALUES (%s, %s, %s, %s, %s)
        '''
        
        session_id = int(line[8])
        item_in_session = int(line[3])
        artist = line[0]
        song = line[9]
        length = float(line[5])

        session.execute(
            query,
            (
                session_id,
                item_in_session,
                artist,
                song,
                length
            )
        )

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Narrative 1st query description: Give me the artist, song title and song's length
# in the music app history that was heard during sessionId = 338, and itemInSession = 4
rows = session.execute('''
    SELECT artist, song, length
    FROM musics_q1
    WHERE session_id = 338
    AND item_in_session = 4
''')
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [14]:
# This data frame should match with CQL query result above.
df[
    (df['sessionId'] == 338) & (df['itemInSession'] == 4)
][['artist', 'song', 'length']]

,artist,song,length
964,Faithless,Music Matters (Mark Knight Dub),495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
second_query_columns = [
    Column(column_name, is_primary_key)
    for (column_name, is_primary_key)
    in [
        ('userId'       , True),
        ('sessionId'    , True),
        ('artist'       , False),
        ('song'         , False),
        ('firstName'    , False),
        ('lastName'     , False),
        ('itemInSession', False)
    ]
]

try:
    validate_column_uniqueness(df, second_query_columns)
except AssertionError as e:
    primary_column_names = [column.name for column in second_query_columns if column.is_primary_key]
    print(f'This is expected since primary columns {primary_column_names} still cause duplication.')
    print('\n')
    print(f'Here is a full error message: {e}')

This is expected since primary columns ['userId', 'sessionId'] still cause duplication.


Here is a full error message: There are 101 duplications when columns = ['userId', 'sessionId', 'artist', 'song', 'firstName', 'lastName', 'itemInSession'] with primary key of ['userId', 'sessionId']


In [16]:
# since we will sort `song` by `itemInSession`,
# let's check if additional primary key `itemInSession` satisfied the second table uniqueness

second_query_columns = [
    Column(column_name, is_primary_key)
    for (column_name, is_primary_key)
    in [
        ('userId'       , True),
        ('sessionId'    , True),
        ('itemInSession', True),
        ('artist'       , False),
        ('song'         , False),
        ('firstName'    , False),
        ('lastName'     , False)
    ]
]

validate_column_uniqueness(df, second_query_columns)

In [17]:
session.execute('''
    CREATE TABLE IF NOT EXISTS musics_q2 (
        user_id         INT,
        session_id      INT,
        item_in_session INT,
        artist          TEXT,
        song            TEXT,
        first_name      TEXT,
        last_name       TEXT,
        PRIMARY KEY (user_id, session_id, item_in_session)
    )
''')

In [18]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user_id = int(line[10])
        session_id = int(line[8])
        item_in_session = int(line[3])
        artist = line[0]
        song = line[9]
        first_name = line[1]
        last_name = line[4]
        
        query = '''
            INSERT INTO musics_q2 (
                user_id,
                session_id,
                item_in_session,
                artist,
                song,
                first_name,
                last_name
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        '''
        
        session.execute(
            query,
            (
                user_id,
                session_id,
                item_in_session,
                artist,
                song,
                first_name,
                last_name
            )
        )

In [19]:
# Narrative 2nd query description: Give me only the following: name of artist,
# song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

rows = session.execute('''
    SELECT artist, song, first_name, last_name
    FROM musics_q2
    WHERE user_id = 10
    AND session_id = 182
    ORDER BY item_in_session
''')
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [20]:
# This data frame should match with CQL query result above.
df[
    (df['userId'] == 10) & (df['sessionId'] == 182)
][
    ['artist', 'song', 'firstName', 'lastName', 'itemInSession']
].sort_values(
    by='itemInSession'
)[['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
151,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
152,Three Drives,Greece 2000,Sylvie,Cruz
153,Sebastien Tellier,Kilometer,Sylvie,Cruz
154,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [21]:
third_query_columns = [
    Column(column_name, is_primary_key)
    for (column_name, is_primary_key)
    in [
        ('song', True),
        ('firstName', False),
        ('lastName', False)
    ]
]

try:
    validate_column_uniqueness(df, third_query_columns)
except AssertionError as e:
    primary_column_names = [column.name for column in third_query_columns if column.is_primary_key]
    print(f'This is expected since primary columns {primary_column_names} still cause duplication.')
    print('\n')
    print(f'Here is a full error message: {e}')

This is expected since primary columns ['song'] still cause duplication.


Here is a full error message: There are 9 duplications when columns = ['song', 'firstName', 'lastName'] with primary key of ['song']


In [22]:
third_query_columns = [
    Column(column_name, is_primary_key)
    for (column_name, is_primary_key)
    in [
        ('song', True),
        ('sessionId', True),
        ('firstName', False),
        ('lastName', False)
    ]
]

validate_column_uniqueness(df, third_query_columns)

In [23]:
session.execute('''
    CREATE TABLE IF NOT EXISTS musics_q3 (
        song            TEXT,
        session_id      INT,
        first_name      TEXT,
        last_name       TEXT,
        PRIMARY KEY (song, session_id)
    )
''') 

In [24]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        song = line[9]
        session_id = int(line[8])
        first_name = line[1]
        last_name = line[4]
        
        query = '''
            INSERT INTO musics_q3 (
                song,
                session_id,
                first_name,
                last_name
            )
            VALUES (%s, %s, %s, %s)
        '''
        
        session.execute(
            query,
            (
                song,
                session_id,
                first_name,
                last_name
            )
        )

In [25]:
# Narrative 3rd query description: Give me every user name (first and last)
# in my music app history who listened to the song 'All Hands Against His Own'

rows = session.execute('''
    SELECT first_name, last_name
    FROM musics_q3
    WHERE song = 'All Hands Against His Own'
''')
for row in rows:
    print(row.first_name, row.last_name)

Sara Johnson


In [26]:
# This data frame should match with CQL query result above.
df[df['song'] == 'All Hands Against His Own'][['firstName', 'lastName']]

,firstName,lastName
219,Sara,Johnson


### Drop the tables before closing out the sessions

In [27]:
for table_name in ['musics_q1', 'musics_q2', 'musics_q3']:
    session.execute(f'''
        DROP TABLE IF EXISTS {table_name}
    ''')

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()